In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np
import scipy
import os
import bilby
import pandas as pd

#####################################################
#This section contains priors, labels for output files 
# and directories and columns and no. of fourier coefficents to be used
label = 'data1'
outdir = './data1'
file_names = ['data1.txt']
flags = ['red'] #needed to combine multiple data
usecols = (0,1,2) #columns to be used
nr=6 #no. of fourier coefficients

#priors
sine_signal=False
#Model Parameters
A=[0,5]  #sine amplitude
PHI=[0,2*np.pi] #phase angle
T0=[0]  #prior of time period of sine signal #max lim is from the data span itself
EFAC=[0.1,4]
EQUAD=[0,4]
OFFSET=[-5,15] #offset from x axis
logPSI=[-5,20] #log priors of psi fourier coefficents
#Noise model
#power law will be a straight line in log log plot
logC=[0,20]  #intercept of the straight line
alpha=[-10,10] #slope
#Output parameters
#parameters for bilby to run the bayesian
sampler="dynesty"
resume=True
npoints=1000
dlogz=1
walks=10
plot=True
########################################################
#A directory is created with name outdir defined above
if not os.path.exists(outdir):
    os.makedirs(outdir)
#loading data and creating a pandas data frame and sort yhe data in MJD
data = dict()
data['Mag'] = np.array([])
data['Magerr'] = np.array([])
data['MJD'] = np.array([])
data['Flag'] = np.array([])
for ii, file_name in enumerate(file_names):
    raw_data = np.loadtxt(file_name,usecols=usecols,delimiter=' ')
    data['Mag']=np.append( data['Mag'], raw_data[:,1] )
    data['Magerr']=np.append( data['Magerr'], raw_data[:,2] )
    data['MJD']=np.append( data['MJD'], raw_data[:,0] )
    data['Flag']=np.append( data['Flag'], np.repeat(flags[ii],len(raw_data[:,1])) )
data = pd.DataFrame.from_dict(data)
data=data.sort_values(by=['MJD']);

#Finding the max data span of the data
Tdtspn= np.max(data["MJD"]) - np.min(data["MJD"])
T0.append(Tdtspn)

#Creating the Fpi matrix defined Abhimanyu's Lentati 2013 document
Fpi=[]
logf=[]
for p in range(2*nr):
    if p<nr:
        tmp=list((1/Tdtspn)*np.sin((2*np.pi*(p+1)*np.array(data["MJD"]))/Tdtspn))
        Fpi.append(tmp)
        logf.append(np.log((2*np.pi*(p+1))/Tdtspn))
    else:
        tmp=list((1/Tdtspn)*np.cos((2*np.pi*(p+1-nr)*np.array(data["MJD"]))/Tdtspn))
        Fpi.append(tmp)
        logf.append(np.log((2*np.pi*(p+1-nr))/Tdtspn))
logf=np.array(logf)
Fpi=np.array(Fpi)
#Fjq is the transpose of Fpi
Fjq=np.transpose(Fpi)

#priors are defined below
priors = dict()
# for psi in range(nr):
#     priors['logPSI-'+str(psi)] = bilby.core.prior.Uniform(logPSI[0], logPSI[1], name='logPSI'+str(psi), latex_label=r'$\logpsi_{'+str(psi)+'}$')
priors['logC'] = bilby.core.prior.Uniform(logC[0], logC[1], name='logC', latex_label=r'$\logC$')
priors['alpha'] = bilby.core.prior.Uniform(alpha[0], alpha[1], name='alpha', latex_label=r'$\alpha$')
for flag in flags:
    priors['efac-'+flag] = bilby.core.prior.Uniform(EFAC[0], EFAC[1], name='efac-'+flag, latex_label='$Efac_{'+flag+'}$')
    priors['equad-'+flag] = bilby.core.prior.Uniform(EQUAD[0], EQUAD[1], name='equad-'+flag, latex_label='$Equad_{'+flag+'}$')
    priors['OFFSET-'+flag] = bilby.core.prior.Uniform(OFFSET[0], OFFSET[1], name='OFFSET-'+flag, latex_label='$offset_{'+flag+'}$')
if sine_signal == True:
    priors['A'] = bilby.core.prior.Uniform(A[0], A[1], name='A', latex_label='$A$ [flux]')
    priors['PHI'] = bilby.core.prior.Uniform(PHI[0], PHI[1],boundary='periodic', name='PHI', latex_label='$\\phi$ [radian]')
    priors['T0'] = bilby.core.prior.Uniform(T0[0], T0[1], name='T0', latex_label='$T_{0}$ [yr]')
parameters = dict.fromkeys(priors.keys())

        
#defined a class to calculate bilby likelihood
#flag is introduced to accomodate different efac equad and offset 
#for different data sets incase we cobine different data sets 
#which are identified by different flags for different datasets
class MultidimGaussianLikelihood(bilby.Likelihood):
    """
        A multivariate Gaussian likelihood

        """
    def __init__(self, data, parameters):
        self._marginalized_parameters = []    
        self.data = data
        self.N = len(data['MJD'])
        self.parameters = parameters


        
        self._efac_array = np.empty(self.N)
        self._equad_array = np.empty(self.N)
        self._offset_array = np.empty(self.N)
        

        self.idx = dict()
        for flag in self.data['Flag']:
            self.idx[flag] = data.index[data['Flag']==flag]
        
#         self.tauij = np.abs( np.repeat(np.asarray(self.data['MJD'])[np.newaxis],self.N,axis=0) - \
#           np.repeat(np.asarray(self.data['MJD'])[np.newaxis].T,self.N,axis=1) )
        
    def map_params_to_array(self):
        
        for flag in self.data['Flag']:
            
            self._efac_array[self.idx[flag]] = self.parameters['efac-'+flag]
            self._equad_array[self.idx[flag]] = self.parameters['equad-'+flag]
            self._offset_array[self.idx[flag]] = self.parameters['OFFSET-'+flag]
#   Likelihood is defined as below
    def log_likelihood(self):
        
        self.map_params_to_array()
#         r1i = np.asarray(self.data['Mag'])[np.newaxis] - np.asarray(self._signal_model())[np.newaxis] - np.asarray(self._offset_array)[np.newaxis]
#        Most of the terms are defined with same names as given in Abhimanyu's Lenetati 2013 document
#        r1i means 1 row i columns
#        r1i = yi - fi
#        fi is sine function or in simple case only const offset
        r1i = np.asarray(self.data['Mag'])[np.newaxis] - np.asarray(self._offset_array)[np.newaxis]
        rj1 = np.transpose(r1i)
#         Defining white noise diagonal matrix
#we need inverse of that matrix and logarithm det of N for further calculation
        Nijtemp = (np.array(self._efac_array)*np.array(self.data['Magerr']))**2+np.array(self._equad_array)**2
        Nij_1temp=np.reciprocal(Nijtemp)
        Nij_1 = np.diag(np.reciprocal(Nijtemp))
        logdetN = np.sum(np.log(Nijtemp))
#         Defining dp1 and its inverse dpq
#        Matrix notation is slightly wrong in Abhimanyu's document
        dp1temp = Fpi*(Nij_1temp[np.newaxis])
        dp1temp1 = dp1temp*r1i
        d1q = np.sum(dp1temp1,axis=1)[np.newaxis]
        dp1 = np.transpose(d1q)

        riNij_1rj = np.sum((r1i[0]**2)/Nijtemp)

#         psitemp=[]
#         psitemp=list(self.parameters.values())[0:nr]
#         psitemp=2*psitemp
#         psitemp=np.array(psitemp)

#       defining diagonal matrix of psi fourier coefficients and log det 
        psitemp=self.parameters['logC']+self.parameters['alpha']*logf
        psitemp1=np.e**(psitemp)
        psi_1=np.diag(np.reciprocal(psitemp1))
        logdetpsi=np.sum(psitemp)

#       Signmapq is Fpi*Nij_1*Fjq so first product is already calculated dp1temp=Fpi*Nij_1
        Sigmapq=np.matmul(dp1temp,Fjq)
        Sigmapq=Sigmapq+psi_1
        Sigmapq_1=np.linalg.pinv(Sigmapq) #pinv is used to avoid the case of singular matrix
        
        d1qSigmapq_1dp1=np.matmul(d1q,Sigmapq_1)
        d1qSigmapq_1dp1=np.matmul(d1qSigmapq_1dp1,dp1)[0][0]

        logdetSigma=np.linalg.slogdet(Sigmapq)[1]
        normal_exponent=riNij_1rj-d1qSigmapq_1dp1
        
        return np.squeeze( -0.5*normal_exponent - 0.5*(  logdetpsi + logdetN + logdetSigma  ))
    
#     def _signal_model(self):
#         result = self.parameters['A']*np.sin(2*np.pi*self.data['MJD']/(self.parameters['T0'])+self.parameters['PHI'])
#         return result
        
lnl = MultidimGaussianLikelihood(data,parameters)

result = bilby.run_sampler(
    likelihood=lnl,
    priors=priors,
    sampler=sampler,
    resume=resume,
    npoints=npoints,
    dlogz=dlogz,
    walks=walks,
    outdir=outdir,
    label=label,
    plot=plot
)
